# Интро

Цель данного проекта создать нейросеть, которая сможет распознавать негативные и токсичные комментарии

В качестве данных для проекта мы будем использовать датасет с Kaggel, который являлся частью конкурса о распознавании токсичных комментариев.


В отличии от того, чтобы делать комментарии на токсичные и не токсичные мы будем использовать шкалу для определения степени токсичности и категории токсичности.


Таким образом каждый сэпмл будет иметь мульти-бинарный таргет. 


Пример:

Допустим у нас есть два сэмпла


1) I hate you 

2) You suck, i'm coming for you


В качестве лейблов для примера у нас будет шкала и категория токсичности (шкала и категории уже взаранее выделены):


1) Низкотоксичный


2) Среднетоксичный


3) Высокотоксичный 


4) Содержит угрозу


Таким образом для сэпмла №1 лейбл будет [0, 1, 0, 0], а для сэпмла №2 [0, 1, 0, 1]



Для достижения цели будет использовать токенизацию слов и эмбеддинг

Токенизирование - это создание числовых лейблов для слов. Каждое слово будет иметь свой лэйбл. Эмбеддинг - это создание значений для каждого токена. К примеру, слово i будет иметь токен 42, а а эмбеддинг к этому слову иметь вектор [0.2, 0.3, 0.9]. Данный вектор будет обозначать степень принадлежности слова к определенной категории (речь не про категории токсичности). На самом деле мы даже не будем знать про категории, которые были созданы в процессе эмбединга, т.к. нейросеть сама будет выделять данные категории.

Далее мы будем использовать преобразованные данные в обучении нейросети, в которой будут использованы LSTM слои, которые хорошо подходят под задачи связанные с последовательностью.

# Импорт библиотек и загрузка данных

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
#не дает модели использовать более 0.333 оперативки, помогает если надо обучать несколько моделей сразу (к примеру, когда
#мы учили несколько моделей сразу играть в игру)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.99)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [3]:
df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
#здесь список категорий для шкалы токсичности
#Пример
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

In [6]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

# Предподготовка данных

In [7]:
from tensorflow.keras.layers import TextVectorization #библиотека для векторизации текстов 

In [8]:
X = df['comment_text']
y = df[df.columns[2:]].values #перевод в values, чтобы произошла конвертация в вектор

In [9]:
MAX_FEATURES = 200000 # количество слов в словаре, который мы создаем с помощью наших комментов
#В большинстве случаев, чем больше количество слов, выделяемых нами для кол-ва слов в словаре, тем больше accuracy модели

In [10]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800, #максимальная длина предложения в токенах (1 токен - 1 слово)
                               #всего будет 1800 значений в токенизированном предложении
                               #если значений меньше 1800, то они заполняются нулями. Это нужно, чтобы
                               #на вход в неросеть у каждого сэмпла была одинаковая форма
                               output_mode='int') #каждое слово будет конвертировано в int

In [11]:
vectorizer.adapt(X.values) #обучение векторизатора нашим словарем

In [12]:
#пример токенизации 
vectorizer('Hello world, life is great')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([286, 261, 305,   9, 275], dtype=int64)>

In [13]:
vectorized_text = vectorizer(X.values) #векторизируем весь текст (все комменты) полностью 

In [14]:
#пайплайн для предобработки данных
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)) #создание датасета
dataset = dataset.cache() #кэширование данных для лучшей производительности
dataset = dataset.shuffle(160000) #перемешивание данных. 160000 - размер буфера
dataset = dataset.batch(16) #Определяем размер батча. В каждом батче будет 16 сэпмлов
dataset = dataset.prefetch(8) # 

In [15]:
dataset.as_numpy_iterator().next() #здесь представлен один батч. В первом array идут X, во втором y

(array([[33109,  3321,    82, ...,     0,     0,     0],
        [   40,   418,     4, ...,     0,     0,     0],
        [  130,    65,    14, ...,     0,     0,     0],
        ...,
        [    8,  1564,   194, ...,     0,     0,     0],
        [65462,    28,  1441, ...,     0,     0,     0],
        [    7,   278,    21, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [16]:
train = dataset.take(int(len(dataset)*.7)) #для трейна берем 70% датасета 
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2)) #пропускаем 70% и берем 20% для валидации
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1)) #пропускаем 90% и берем 10%

# Создание модели

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [18]:
model = Sequential()
# Сначала создаем embedding слой
model.add(Embedding(MAX_FEATURES+1, 32)) #первый арг - сколько типов эмбеддинга можно сделать максимум. Один эмбеддинг
#на одно слово. 32 - это размер каждого эмбеддинга (сколько признаков модель может приписать слову)
# Созданиие LSTM слоя
#Создаем Bidirectional слой, потому что нам важен порядок как слева на право, так и справа на лево. Объясним:
# Обычный LSTM слой "читает" слова в одном направлении (пр. слева на право). И тогда для предложения i dont hate you
#нейросеть прочитает последенее слово hate и сделает вывод об негативности данного предложения. При Bidirectional функции
# нейросеть "заметит", то есть придаст вес слову don't и таким образом предложение потеряет негативынй смысл
#bidirectional помогает в заданиях рабоыт с языком.
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Слои для обучения 
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Слой для аутпута
model.add(Dense(6, activation='sigmoid'))

In [19]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
#используем BinaryCrossentropy т.к. у нас мы грубо говоря запускаем 6 бинарных классификаторов одновременно

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 256)               33024     
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 6)                 774       
                                                      

In [70]:
# history = model.fit(train, epochs=1, validation_data=val) #обучение модели

In [20]:
model = tf.keras.models.load_model('toxicity.h5') #lstm слои обычно обучаются достаточно долгое время, поэтому загрузим модель

# Создание прогноза

In [22]:
#для начала нужно токенизировать текст уже обученным токенайзером
input_text = vectorizer('Eat my ass you stupid fuck')

In [23]:
model.predict(np.expand_dims(input_text, 0))
#np.expand_dims(input_text, 0) тоже самое, что np.array([input_text])
#Нам нужно это для того, чтобы соблюсти форму на входе в нейросеть

1/1 [==============================] - 10s 10s/step


array([[0.99877197, 0.3512748 , 0.97631073, 0.04549523, 0.89834106,
        0.23001862]], dtype=float32)

В итоге получили array в котором каждое значение являятся вероятностью для сэпмла принадллежать к категории:

In [77]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

Допустим барьером для токсичности будет вероятность сэмпла принадлежать к категории выше 0.5, тогда можно выявлять сразу же  с помощью:

In [24]:
(model.predict(np.expand_dims(input_text, 0)) > 0.5).astype('int')

1/1 [==============================] - 0s 72ms/step


array([[1, 0, 1, 0, 1, 0]])

# Оценка модели

In [80]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [81]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [82]:
for batch in test.as_numpy_iterator(): #т.к. наши данные поделены на батчи, то для каждого батча...
    # Распаковываем батч
    X_true, y_true = batch
    # Делаем прогноз
    yhat = model.predict(X_true)
    
    # Конвертируем в 1-D array  
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 120ms/step


1/1 [==============================] - 0s 120ms/step


1/1 [==============================] - 0s 119ms/step


1/1 [==============================] - 0s 108ms/step


In [83]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.8229508399963379, Recall:0.7015092372894287, Accuracy:0.46840521693229675


В итоге мы получили не самый высокие результаты, однако мы могли бы улучшить их увеличив кол-во эпох, т.к. функция потерь продолжала падать.